# Create Dataloader

In [1]:
import os
import pandas as pd
from dotmap import DotMap

from utils.io import load_yaml
from utils.io import load_pickle, save_pickle
from utils.transforms import anyDictListToCompose
from lightning_data_modules import VideoDataModule

server = 'local'
server = DotMap(load_yaml('../config/servers.yml')[server])

meta_df_name = 'cholec80_retransforms.pkl'
meta_df = pd.read_pickle(os.path.join(server.config.location, meta_df_name))
meta_df.aug_transforms = None  # no transforms for evaluation

# load cholec80
prefix = server.database.location
clip_length_in_frames = 2
frames_between_clips = 1
frame_rate = 5
train_split = 0.5
batch_size = 1
num_workers = 8
random_state = 42

# None if not initialized
test_md_name = 'cholec80_homography_regression_discrepancy_test_md_frame_rate_{}.pkl'.format(frame_rate)

try:
    test_md = load_pickle(os.path.join(server.config.location, test_md_name))
except:
    test_md = None

dm = VideoDataModule(
    meta_df,
    prefix=prefix,
    clip_length_in_frames=clip_length_in_frames,
    frames_between_clips=frames_between_clips   ,
    frame_rate=frame_rate,
    train_split=train_split,
    batch_size=batch_size,
    num_workers=num_workers,
    random_state=random_state,
    test_metadata=test_md
)

_, _, test_md = dm.setup('test')

# store metadata
if test_md:
    save_pickle(os.path.join(server.config.location, test_md_name), test_md)

In [19]:
import pandas as pd

df = pd.read_pickle('../config/cholec80_transforms.pkl')
df.pre_transforms = df.pre_transforms.apply(lambda x: [
    x[0],
    {
        'module': 'torchvision.transforms',
        'type': 'Resize',
        'kwargs': {'size': [int(y/2) for y in x[1]['kwargs']['size']]}
    }
])
df.to_pickle('../config/cholec80_retransforms.pkl')


# Evaluate Feature-based Homography Estimation

In [12]:
import cv2
import numpy as np
from tqdm import tqdm
import torch
from kornia import warp_perspective, tensor_to_image

from lightning_modules import DeepImageHomographyEstimationModuleBackbone
from utils.processing import four_point_homography_to_matrix, image_edges
from utils.processing import FeatureHomographyEstimation
from utils.viz import yt_alpha_blend
from utils.io import generate_path

# feature-based model creationg stage
fd = cv2.SIFT_create()
fh = FeatureHomographyEstimation(fd)

# deep creationg stage
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone_search/version_8'
configs = load_yaml(os.path.join(prefix, 'config.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=99-step=44099.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()
model.freeze()

# log dataframe
log_df = pd.DataFrame(columns=['frame_rate', 'video_fps', 'video_idx', 'idx', 'duv'])

# data iterator
test_dl = dm.test_dataloader()

hist = []

max_cnt = 3000
cnt = 0

for batch in tqdm(test_dl):
    if cnt > max_cnt:
        break
    cnt += 1
    # deep
    batch = dm.transfer_batch_to_device(batch, device)
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    img_deep, wrp_deep = vid[0,0].unsqueeze(0), vid[0,1].unsqueeze(0)

    duv_deep = model(img_deep, wrp_deep)
    duv_deep *= -1

    uv_deep = image_edges(img_deep)
    H_deep = four_point_homography_to_matrix(uv_deep, duv_deep)

    # classical
    batch = dm.transfer_batch_to_device(batch, 'cpu')
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    img, wrp = tensor_to_image(vid[0,0]), tensor_to_image(vid[0,1])

    H, duv = fh((img*255).astype(np.uint8), (wrp*255).astype(np.uint8))

    if H is not None:
        mpd = np.linalg.norm(duv - duv_deep.cpu().numpy().squeeze(), axis=1).mean()
        log_df = log_df.append({
            'frame_rate': frame_rate.numpy(), 'video_fps': np.array([x.numpy() for x in video_fps]), 'video_idx': video_idx.numpy(), 'idx': idx.numpy(), 'duv': duv, 'mpd': mpd
        }, ignore_index=True)

        hist.append(mpd)
        # # deep
        # wrp_deep_reg = warp_perspective(img_deep, H_deep, (img_deep.shape[-2], img_deep.shape[-1]))

        # blend_deep = yt_alpha_blend(wrp_deep, wrp_deep_reg)

        # blend_deep = tensor_to_image(blend_deep)
        # cv2.imshow('blend_deep', blend_deep)

        # # classical
        # wrp_reg = cv2.warpPerspective(img, H, (wrp.shape[1], wrp.shape[0]))

        # blend = yt_alpha_blend(wrp, wrp_reg)

        # cv2.imshow('blend', blend)
        # cv2.waitKey()

log_path = os.path.join(server.logging.location, 'homography_regression_discrepancy')
generate_path(log_path)
log_df.to_pickle(os.path.join(log_path, 'feature_based_frame_rate_{}.pkl'.format(frame_rate.item())))

  0%|          | 0/89715 [00:00<?, ?it/s]Running with CUDA backend.
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:1

In [2]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from kornia import tensor_to_image

log_path = os.path.join(server.logging.location, 'homography_regression_discrepancy')
frame_rate = 5
df = pd.read_pickle(os.path.join(log_path, 'feature_based_frame_rate_{}.pkl'.format(frame_rate)))

ds = dm._test_set

for _, row in df[df.mpd > 100].iterrows():
    
    batch = ds[row.idx.item()]
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    img, wrp = tensor_to_image(vid[0]), tensor_to_image(vid[1])

    cv2.imshow('img', img)
    cv2.imshow('wrp', wrp)
    cv2.waitKey()




# 

# hist = np.array(hist)
# print(hist.mean())
# print(hist.std())

# plt.hist(hist, bins=50)
# plt.show()

/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


# Evaluate Deep Homography Estimation

In [11]:
import cv2
import torch
from kornia import warp_perspective, tensor_to_image

from lightning_modules import DeepImageHomographyEstimationModuleBackbone
from utils.processing import four_point_homography_to_matrix, image_edges
from utils.viz import yt_alpha_blend
from utils.io import generate_path

# model creationg stage
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone_search/version_6'
configs = load_yaml(os.path.join(prefix, 'config.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=52-step=46692.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()
model.freeze()

# log dataframe
log_df = pd.DataFrame(columns=['frame_rate', 'video_fps', 'video_idx', 'idx', 'duv'])

# data iterator
test_dl = dm.test_dataloader()

for batch in test_dl:
    batch = dm.transfer_batch_to_device(batch, device)
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    img, wrp = vid[0,0].unsqueeze(0), vid[0,1].unsqueeze(0)

    duv = model(img, wrp)
    duv *= -1

    log_df = log_df.append({
        'frame_rate': frame_rate.cpu().numpy(), 'video_fps': np.array([x.cpu().numpy() for x in video_fps]), 'video_idx': video_idx.cpu().numpy(), 'idx': idx.cpu().numpy(), 'duv': duv.cpu().numpy()
    }, ignore_index=True)

    # uv = image_edges(img)
    # H = four_point_homography_to_matrix(uv, duv)

    # wrp_reg = warp_perspective(img, H, (img.shape[-2], img.shape[-1]))

    # blend = yt_alpha_blend(wrp, wrp_reg)

    # blend = tensor_to_image(blend)
    # cv2.imshow('blend', blend)
    # cv2.waitKey()

log_path = os.path.join(server.logging.location, 'homography_regression_discrepancy')
generate_path(log_path)
log_df.to_pickle(os.path.join(log_path, 'deep_frame_rate_{}.pkl'.format(frame_rate.item())))

# iterate through test set

# estimate homography, store duv, idx -> mangage to load frame by index

Running with CUDA backend.
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/kornia/geometry/transform/imgwarp.py:98: UserWarning: The align_corners default value has been changed. By default now is set True in order to match cv2.warpPerspective. In case you want to keep your previous behaviour set it to False. This warning will disappear in kornia > v0.6.
  warnings.warn(message)


In [4]:
feature_df = pd.read_pickle('/media/martin/Samsung_T5/logs/homography_regression_discrepancy/feature_based_frame_rate_5.pkl')
deep_df = pd.read_pickle('/media/martin/Samsung_T5/logs/homography_regression_discrepancy/deep_frame_rate_5.pkl')

# diff = feature_df.duv - deep_df.duv

# print(diff)

0      [[[-35.307350158691406, 25.879680633544922], [...
1      [[[-25.74515390396118, 23.259612321853638], [-...
2      [[[-34.93977928161621, 28.84478187561035], [-0...
3      [[[-20.24051344394684, 21.259456872940063], [-...
4      [[[-35.004459381103516, 36.209375500679016], [...
                             ...                        
295    [[[-5.1263673305511475, -6.074093461036682], [...
296    [[[-10.244562864303589, -5.198544144630432], [...
297    [[[-8.791977047920227, -5.269481897354126], [5...
298    [[[-4.958159148693085, -5.206058144569397], [1...
299    [[[-6.0587605237960815, -4.56786236166954], [-...
Name: duv, Length: 300, dtype: object
